# Create the Chatbot

In [10]:
#ChatBot
import chatterbot
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import re
import pandas as pd

#Topic Recognition
import tensorflow as tf
import numpy as np
import spacy

#API
#import infermedica_api
#infermedica_api.configure(app_id='XX', app_key='XXX')
#api = infermedica_api.get_api()

In [2]:
chatc2 = pd.read_csv('../data/scrap-counsel-chat.txt') 

chatc2.drop(['Unnamed: 0','questionID','questionLink','therapistInfo','therapistURL','upvotes','views','split','topic','questionTitle'], axis=1, inplace=True)

corpus = chatc2.answerText
answers=[]
for sentence in corpus:
    answers.append(re.split('\.!?', sentence))

answer_lst = [answer for sublist in answers for answer in sublist]

movie = pd.read_csv('../data/movie_lines.txt', sep='|', names=['1', '2', '3', '4', 'MovieLines'])   
movie = list(movie.MovieLines.astype('str'))

In [3]:
bot = ChatBot(name='Hoa',         #let's find him a cute name !
              read_only=False,     #so it can learn after training
              storage_adapter='chatterbot.storage.SQLStorageAdapter',
              logic_adapters=[{'import_path': 'chatterbot.logic.BestMatch',
                               'default_response': 'Can you tell me more about how you feel? 💬',
                               'maximum_similarity_threshold': 0.9}])
                               #'statement_comparison_function': 'chatterbot.comparisons.LevenshteinDistance'
                               #'response_selection_method': 'chatterbot.response_selection.get_first_response'

In [4]:
list_trainer = ListTrainer(bot)
"""from chatterbot.trainers import ChatterBotCorpusTrainer
corpus_trainer = ChatterBotCorpusTrainer(bot)

files = ['conversations.yml', 'greetings.yml', 'health.yml']
for file in files:
    corpus_trainer.train('C:\\Users\\sarah\\anaconda3\\Lib\\site-packages\\chatterbot_corpus\\data\\english\\'+file)
list_trainer.train(movie)

list_trainer.train(answer_lst)"""

"from chatterbot.trainers import ChatterBotCorpusTrainer\ncorpus_trainer = ChatterBotCorpusTrainer(bot)\n\nfiles = ['conversations.yml', 'greetings.yml', 'health.yml']\nfor file in files:\n    corpus_trainer.train('C:\\Users\\sarah\\anaconda3\\Lib\\site-packages\\chatterbot_corpus\\data\\english\\'+file)\nlist_trainer.train(movie)\n\nlist_trainer.train(answer_lst)"

In [5]:
topic_rec = tf.keras.models.load_model('topic_text_1000.model')

In [6]:
#I'll use spacy as it seemed like a good option to lemmatize with the appropriate pos tag, detects pronouns and superlative 
#forms of words.
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

#Clean the text:
def clean_input(s):
    s = s.lower()                   #remove caps to avoid double words
    s = re.sub('[\W\d]', ' ', s)    #remove special signs, punctuation, numbers
    s = re.sub(' +', ' ', s)        #remove excessive spaces
    s = s.strip()                   #remove first and last spaces
    return s

def spacy_lem(l):
    doc = nlp(l)
    return [token.lemma_ for token in doc]

word_features_topic = pd.read_csv('../data/word_features_topic.csv')
word_features = list(word_features_topic['0'])

def vectorize_input(l):
    new_list = []
    for word in word_features:
        if word in l:
            new_list.append(1)
        else:
            new_list.append(0)
    return new_list

topics = {'SPIRITUALITY':0,
          'COUNSELING':1,
          'WORKPLACE':2,
          'FAMILY':3,
          'RELATIONSHIPS':4,
          'SLEEP':5,
          'BEHAVIOR':6,
          'SEXUALITY':7,
          'SELF_ESTEEM':8,
          'GRIEF':9,
          'TRAUMA':10,
          'STRESS':11,
          'EATING_DISORDERS':12,
          'ADDICTION':13,
          'DEPRESSION':14,
          'LGBTQ':15,
          'DOMESTIC_VIOLENCE':16,
          'SELF_HARM':17}

def topic_recognizer(raw_text):

    vect_text = vectorize_input(spacy_lem(clean_input(raw_text)))
#    vect_text = np.array(vect_text)
    pred = topic_rec.predict([vect_text])

    recognized_topics = []
    
    if np.max(pred) > 0.5:
        for key in topics: 
            if np.argmax(pred) == topics[key]:
                recognized_topics.append((key, round((pred[0][np.argmax(pred)])*100, 2)))
    else:
        for p in (-pred).argsort():
            for top in p[:3]:
                for key in topics:
                    if top == topics[key]:
                        recognized_topics.append((key, round((pred[0][top])*100, 2)))

    return recognized_topics

In [7]:
topics_solution = {'SPIRITUALITY':'https://www.mind.org.uk/information-support/your-stories/mind-body-spirit/',
                   'COUNSELING':'https://www.mind.org.uk/information-support/guides-to-support-and-services/abuse/considering-counselling/',
                   'WORKPLACE':'https://www.mind.org.uk/workplace/',
                   'FAMILY':'http://www.mentalhealthcenter.org/how-does-family-life-affect-mental-health/',
                   'RELATIONSHIPS':'https://www.mind.org.uk/information-support/your-stories/managing-mental-health-in-a-relationship/',
                   'SLEEP': 'c_721',
                   'BEHAVIOR':'https://www.mind.org.uk/information-support/types-of-mental-health-problems/mental-health-problems-introduction/difficult-feelings-behaviours/',
                   'SEXUALITY':'https://www.healthdirect.gov.au/sexuality-mental-health',
                   'SELF_ESTEEM':'https://www.mind.org.uk/information-support/types-of-mental-health-problems/self-esteem/about-self-esteem/',
                   'GRIEF':'https://www.mind.org.uk/information-support/guides-to-support-and-services/bereavement/experiences-of-grief/',
                   'TRAUMA':'c_299',
                   'STRESS':'c_117',
                   'EATING_DISORDERS':'https://www.mind.org.uk/information-support/types-of-mental-health-problems/eating-problems/treatment-support/',
                   'ADDICTION':'http://www.beatingaddictions.co.uk/',
                   'DEPRESSION':'c_26',
                   'LGBTQ':'https://www.mind.org.uk/information-support/tips-for-everyday-living/lgbtiqplus-mental-health/about-lgbtiqplus-mental-health/',
                   'DOMESTIC_VIOLENCE':'https://www.mind.org.uk/information-support/guides-to-support-and-services/abuse/domestic-violence/, https://www.mind.org.uk/information-support/guides-to-support-and-services/abuse/about-this-resource/',
                   'SELF_HARM':'https://www.mind.org.uk/information-support/types-of-mental-health-problems/self-harm/about-self-harm/, https://www.mind.org.uk/information-support/types-of-mental-health-problems/suicidal-feelings/about-suicidal-feelings/'}

In [8]:
topic_recognizer('My spouse visited the red light district a couple of years back in Amsterdam. After he got back from the trip he got me tested for stds. In the aftermath he has provided multiple versions of what happened, from having a total blackout to being touched by a prostitute. How do I know if he had sex?')

[('SEXUALITY', 99.88)]

In [12]:
print('\033[1mDISCLAIMER\033[0m: \nYou are about to talk with a generated chatbot, made to provide you with the most appropriate solution.',
      'Some answer might not always be the best and it is always good to double check with a specialist. \n\n')
print("\033[1mHoa\033[0m: Hi! My name is Hoa, and I'm here to help you find a solution. Please, can you describe your issue ?")
print('If you wanna exit this conversation, then type "exit".\n')

conversation = True            #To keep the conversation running
recognized_topics = []
user_input = input('\n\033[1mYou\033[0m: ')

while conversation == True:
    
    user_corpus = ''               #Will be used for topic recognition
    
    if user_input != 'exit':
        
        user_corpus = user_corpus + ' ' + user_input
        
        while len(user_corpus) < 300:  #300 characters, to make sure I have enough key words for the topic recognition
            print('\n\033[1mHoa\033[0m:', bot.get_response(user_input))
            user_input = input('\n\033[1mYou\033[0m: ')
            user_corpus = user_corpus + ' ' + user_input
        
        recognized_topics = topic_recognizer(user_corpus)
        print('\n\033[1mHoa\033[0m:', 'Sorry to hear that..')

        for tple in recognized_topics: 
            if topics_solution[tple[0]].startswith('c') == True:
                response = api.condition_details(topics_solution[tple[0]])
                print('\n\033[1mHoa\033[0m:', 'I understand you suffer from', response.common_name)
                print('\033[1mHoa\033[0m:', 'Here is my advice :', response.extras['hint'])
                
            else:
                print('\n\033[1mHoa\033[0m:', 'I understand you have issues with', tple[0].lower())
                print('\033[1mHoa\033[0m:', 'This website has meaningful ressources, try calling their helpline as well :', topics_solution[tple[0]])
        
        user_input = input('\n\033[1mYou\033[0m: ')
        
    else:
        conversation = False

#End of conversation
print('\n\033[1mHoa\033[0m:', "Mental health is serious. You're not wasting anyone's time. Take care my friend ! 🤖")



















DISCLAIMER: 
You are about to talk with a generated chatbot, made to provide you with the most appropriate solution. Some answer might not always be the best and it is always good to double check with a specialist. 


Hoa: Hi! My name is Hoa, and I'm here to help you find a solution. Please, can you describe your issue ?
If you wanna exit this conversation, then type "exit".


You: hello

Hoa: Hi

You: i need advice

Hoa: Okay, come on down.

You: can you help me?

Hoa: No.

You: why not?

Hoa: Bianca Stratford. Sophomore. Don't even think about it

You: ok sorry

Hoa: Get this. We ain't partners. We ain't brothers. We ain't friends. I'm puttin' you down and keepin' you down until Ganz is locked up or dead. And if Ganz gets away, you're gonna be sorry we ever met.

You: i have a headache

Hoa: Oh, you got a headache!

You: i am stressed about my presentation

Hoa: Good God... they're meat eaters, Roy, just drop the shit in there!

You: i have anxiety

Hoa: Did you take medicine?

You: 